<a href="https://colab.research.google.com/github/anthony-pipeline/HyperSkill/blob/main/pre_trained_rubert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from random import randint
import torch
from torch.utils.data import (TensorDataset,
                              DataLoader,
                              RandomSampler)

from keras.preprocessing.sequence import pad_sequences

from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AutoTokenizer
from transformers import AdamW

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

False


# Подготовка данных

In [5]:
df = pd.read_csv("nottoolong_reviews.csv")

In [6]:
df.head()

,feedback,rating
0,Был в отделении Абсолют Банка на Кутузовском п...,POSITIVE
1,"У банка не работало приложение с 1 по 4 мая, у...",NEGATIVE
2,Хочу поделиться впечатлением от посещения банк...,POSITIVE
3,Не первый год являюсь партнером Абсолют банка....,POSITIVE
4,Обслуживание в интернет банке не заслуживает н...,NEGATIVE


## Функция для индексации категорий


In [7]:
df.rating.unique()

array(['POSITIVE', 'NEGATIVE', 'NEUTRAL'], dtype=object)

In [8]:
def make_index(x):
  if x == "POSITIVE":
    return 2
  if x == "NEUTRAL":
    return 1
  if x == "NEGATIVE":
    return 0

In [9]:
df["rating"] = df["rating"].apply(lambda x: make_index(x))

In [10]:
df.rating.unique()

array([2, 0, 1])

In [11]:
sentences = df.feedback.values
labels = df.rating.values

# Сначала токенайзер

In [12]:
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'DeepPavlov/rubert-base-cased')

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main


Теперь подготовленный датасет нужно токенизировать. Для этого мы воспользуемся уже предобученным токенизатором, идущим вместе с моделью от DeepPavlov.

Каждый сэмпл мы приведём к виду, который требуется Берту, затем разобъём выборку на обучающую и отложенную, для последующей проверки качества.

Специальные токены Берта:

    [CLS] - начало последовательности
    [SEP] - разделение двух предложений

Мы будем обрамлять наши сэмплы этими токенами.

In [13]:
sentences = ['[CLS] ' + sentence + ' [SEP]' for sentence in sentences]

In [14]:
train_sentences, test_sentences, train_category, test_category = train_test_split(sentences, labels, test_size=0.1)

In [15]:
# Токенизируем обучающую выборку
tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]

In [16]:
print(tokenized_texts[432])

['[CLS]', 'Постоянно', 'отврат', '##ительное', 'обслуживание', 'и', 'отношение', 'к', 'клиентам', '.', 'Раньше', 'у', 'ВТБ', 'был', 'рекламный', 'слоган', '"', 'Дальше', '-', 'Больше', '"', '-', 'банк', 'безусловно', 'продолжает', 'воплощ', '##ать', 'этот', 'слоган', 'в', 'жизнь', '.', 'Ну', 'не', 'нужны', 'банку', 'клиенты', '!', 'Потреб', '##овалось', '15', '.', '11', '.', '19', 'снять', 'с', 'деб', '##етов', '##ой', 'карты', 'незначительную', 'налич', '##ность', ',', 'ни', 'в', 'одном', 'из', 'банкоматов', 'отделения', 'не', 'оказалось', 'налич', '##ности', 'от', 'слова', '"', 'совсем', '"', '(', 'например', 'банком', '##ат', '397', '##11', '##9', ')', ',', 'даже', 'без', 'запроса', 'суммы', '-', 'снятие', 'наличных', 'невозможно', '.', 'Лад', '##но', ',', 'время', '18', ':', '44', 'посетителей', 'в', 'отделении', 'почти', 'нет', ',', 'трое', 'опера', '##цион', '##истов', 'усердно', 'смотрят', 'в', 'свои', 'мониторы', ',', 'отделение', 'работает', 'до', '20', ':', '00', '.', 'Бер', 

In [ ]:
# индексы токенов
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_texts]